# Pythonプログラムの解説

## 参考にしたコード

- ノードの形・色・枠線の変更

https://github.com/cytoscape/cytoscape-automation/blob/master/for-scripters/Python/Overview-of-py4cytoscape.ipynb

- ファイルのインポート

https://github.com/cytoscape/cytoscape-automation/blob/master/for-scripters/Python/importing-data-from-tables.ipynb
https://github.com/cytoscape/cytoscape-automation/blob/master/for-scripters/Python/importing-network-from-table.ipynb

## 実行環境
- Python 3.12.3
- Cytoscape 3.10.3
- py4cytoscape 1.11.0

## Jupyter-Bridge をセットアップ

Cytoscape と Jupyter 環境を橋渡しするための設定（jupyter-bridge）をします。

具体的には、ローカルの Cytoscape とリモートの Notebook の間で通信可能にする JavaScript と Python の仕組みが読み込まれます。



In [1]:
import requests
exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

Requirement already satisfied: py4cytoscape in c:\users\bb38121075\appdata\roaming\python\python310\site-packages (1.12.0)

Requirement already satisfied: pandas in c:\users\bb38121075\appdata\local\programs\python\python310\lib\site-packages (from py4cytoscape) (1.4.3)

Requirement already satisfied: networkx in c:\users\bb38121075\appdata\roaming\python\python310\site-packages (from py4cytoscape) (3.4.2)

Requirement already satisfied: requests in c:\users\bb38121075\appdata\local\programs\python\python310\lib\site-packages (from py4cytoscape) (2.32.3)

Requirement already satisfied: igraph in c:\users\bb38121075\appdata\roaming\python\python310\site-packages (from py4cytoscape) (0.11.8)

...

Requirement already satisfied: idna<4,>=2.5 in c:\users\bb38121075\appdata\local\programs\python\python310\lib\site-packages (from requests->py4cytoscape) (3.3)

Requirement already satisfied: urllib3<3,>=1.21.1 in c:\users\bb38121075\appdata\local\programs\python\python310\lib\site-packages (from requests->py4cytoscape) (1.26.9)

Requirement already satisfied: certifi>=2017.4.17 in c:\users\bb38121075\appdata\local\programs\python\python310\lib\site-packages (from requests->py4cytoscape) (2021.10.8)

Loading Javascript client ... bd03dd2f-1fdc-4e44-8103-f776cfdb2825 on https://jupyter-bridge.cytoscape.org


<IPython.core.display.Javascript object>

## py4cytoscapeをインポート



In [2]:
import py4cytoscape as p4c
p4c.cytoscape_ping()
p4c.cytoscape_version_info()

You are connected to Cytoscape!


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.11.0-SNAPSHOT',
 'automationAPIVersion': '1.12.0',
 'py4cytoscapeVersion': '1.12.0'}

## ネットワーク（sif）、ノード属性（tsv）をインポート

In [3]:
import os
import sys
import pandas as pd

In [5]:
p4c.import_network_from_file('7network.sif')

{'networks': [8298], 'views': [20250]}